# Creating Executable Files in Bash

Often times you'll want to take code you use use repeatedly and bundle them up into functions. Reducing the number of times is
always a good practice when it comes to code. It saves time and reduces chance of mistakes.

Two of the most common methods are functions and aliases. These are often put into one's ~/.bashrc file and are directly in the current shell environment. However sometimes with complicated functions, you might prefer having them as their own commands similar to $\texttt{ls}$ in the $\texttt{\bin}$ directories. These are shell scripts that are designed to be executables, mini-programs. One of the nuances of these shell scripts however is allowing them to make changes to your current shell environment, as when they are called, they open a new "subshell" before returning to the original shell where the executable was initially called upon completion. 

## Subshells

Say we often get stuck within  $\texttt{cd}$ into a really long file path. In this case, our example filepath can be $\texttt{/home/thsiao3/notebooks}$, the home directory for my Jupyter notebooks. Below is a shell script, $\texttt{jump}$ that takes one argument which is a directory in the chain of directories above my current one that it will cd into. 

In [8]:
which jump

/usr/local/bin/jump


In [9]:
cat jump

#! /bin/bash

# original value of Internal Field Separator 
OLDIFS=$IFS 
  
# setting field separator to "/"  
IFS=/ 
  
# converting working path into array of directories in path 
# eg. /my/path/is/like/this 
# into [, my, path, is, like, this] 
path_arr=($PWD)
  
# setting IFS to original value 
IFS=$OLDIFS 
  
pos=-1 
  
# ${path_arr[@]} gives all the values in path_arr 
for dir in "${path_arr[@]}"
do
    # find the number of directories to move up to 
    # reach at target directory  
    pos=$[$pos+1] 
    if [ "$1" = "$dir" ];then
  
        # length of the path_arr 
        dir_in_path=${#path_arr[@]} 
  
        #current working directory 
        cwd=$PWD 
        limit=$[$dir_in_path-$pos-1]
        for ((i=0; i<limit; i++)) 
        do
            echo $i
	    cd .. 
 	    echo $PWD
	done
        break
    fi
done


Let's try using jump to cd into the /home directory, two directories above my current one. 

In [12]:
pwd

/home/thsiao3/notebooks


In [13]:
jump home

0
/home/thsiao3
1
/home


In [14]:
pwd

/home/thsiao3/notebooks


So when I try to $\texttt{cd}$ from a shell script that I set to be executable in my $\texttt{PATH}$ in $\texttt{/usr/local/bin}$, I can see
from print statements that the correct directory is being changed. However, in my original session, my current working directory
is left unchanged. 

This is because when I just run an executable shell script (indicated by the shebang on the first line invoking $\texttt{#! /bin/bash}$), that script is run in another subshell. So the directory is being changed in that subshell, but once the script finishes running, it just returns to the original 

## The $\texttt{source}$ command

The source command takes one argument, a file which in our case is our executable shell script. Source will read and execute commands
in this file in the $\textbf{current}$ shell environment, rather than run it in a new subshell as is default behavior.

In [17]:
pwd

/home/thsiao3/notebooks


In [18]:
source jump home

0
/home/thsiao3
1
/home


In [19]:
pwd

/home


Another way to use source is the '.', they are equivalent. 

In [25]:
cd /home/thsiao3/notebooks
pwd

/home/thsiao3/notebooks


In [26]:
. jump home

0
/home/thsiao3
1
/home


In [27]:
pwd

/home


Sometimes however, it may just be the case that a function in your .bash_profile or .bash_rc is the easiest thing to do. In that
scenario, you do not need to worry about different shells being used to run commands, since functions are run in the 
current shell they are called form.